<h1><div align="center">异步流及 CUDA C/C++ 应用程序的可视化性能分析</div></h1>

![CUDA](./images/CUDA_Logo.jpg)

CUDA工具包附带了 **Nsight Systems**，这是一个功能强大的GUI应用程序，可支持CUDA应用程序的开发。 Nsight Systems为被加速的应用程序生成图形化的活动时间表，其中包含有关CUDA API调用、内核执行、内存活动以及**CUDA流**的使用的详细信息。

在本实验中，您将使用Nsight Systems时间表来指导您优化被加速的应用程序。此外，您还将学习一些中级的CUDA编程技术来支持您的工作：**手动的内存分配和迁移**； **固定**或**页面锁定**的主机内存；以及**非默认并发CUDA流**。

本实验学习课程结束时，我们将为您提供一份评估测试，让您加速和优化一款简单的 n-body 模拟器，这可让您借此展示在本课程学习期间所掌握的技能。若您能够在保持正确性的同时加速模拟器，我们将为您颁发证书以资证明。

---
## 预备知识

为了充分利用本实验，您应该已经能够：

- 编写，编译和运行既调用CPU函数又启动GPU核函数的C / C ++程序。
- 使用执行配置(execution configuration)控制并行线程的层次结构。
- 重构串行循环以在GPU上并行执行其迭代。
- 分配和释放CUDA统一内存。
- 了解统一内存在页面错误和数据迁移方面的行为。
- 使用异步内存预取可以减少页面错误和数据迁移。

## 目标

在完成本练习后，您将能够：

- 使用**Nsight Systems**直观地描述由GPU加速的CUDA应用程序的时间表。
- 使用**Nsight Systems**识别和利用CUDA应用程序中的优化机会。
- 利用CUDA流在被加速的应用程序中并发执行核函数。
- （ **可选的进阶内容** ）使用手动的设备内存分配，包括分配固定的内存，以便在并发CUDA流之间异步传输数据。


---
## 运行Nsight Systems

在此交互式实验环境中，我们设置了一个远程桌面，您可以从您的浏览器访问该桌面，并在其中启动和使用Nsight Systems。

首先，为一个已经存在的向量加法程序创建一个性能报告文件，然后逐步执行一系列步骤，以在Nsight Systems中打开该报告文件，并提供良好的视觉体验。

### 生成报告文件

[`01-vector-add.cu`](../edit/01-vector-add/01-vector-add.cu)（<--单击这个链接以在浏览器中对源文件进行编辑）包含一个正常的矢量加法程序。 使用下方的代码执行单元（通过按CTRL+ENTER，您可以执行它以及本实验中的任何代码执行单元）来编译并运行它。 您应该看到一条表明已成功完成的消息。

In [2]:
!nvcc -o vector-add-no-prefetch 01-vector-add/01-vector-add.cu -run

Success! All values calculated correctly.


接下来，使用`nsys profile --stats = true`创建一个报告文件，您将可以在Nsight Systems可视化分析器中打开该文件。 在这里，我们使用-o标志为报告文件起一个容易记住的文件名：

In [3]:
!nsys profile --stats=true -o vector-add-no-prefetch-report ./vector-add-no-prefetch


**** collection configuration ****
	output_filename = /dli/task/vector-add-no-prefetch-report
	force-overwrite = false
	stop-on-exit = true
	export_sqlite = true
	stats = true
	capture-range = none
	stop-on-range-end = false
	Beta: ftrace events:
	ftrace-keep-user-config = false
	trace-GPU-context-switch = false
	delay = 0 seconds
	duration = 0 seconds
	kill = signal number 15
	inherit-environment = true
	show-output = true
	trace-fork-before-exec = false
	sample_cpu = true
	backtrace_method = LBR
	wait = all
	trace_cublas = false
	trace_cuda = true
	trace_cudnn = false
	trace_nvtx = true
	trace_mpi = false
	trace_openacc = false
	trace_vulkan = false
	trace_opengl = true
	trace_osrt = true
	osrt-threshold = 0 nanoseconds
	cudabacktrace = false
	cudabacktrace-threshold = 0 nanoseconds
	profile_processes = tree
	application command = ./vector-add-no-prefetch
	application arguments = 
	application working directory = /dli/task
	NVTX profiler range trigger = 
	NVTX profiler domain trigge

### 打开远程桌面

运行下一个代码单元格以生成指向远程桌面的URL，您应该通过复制该URL并将其粘贴到新的浏览器选项卡中来打开该URL。 然后，按照notebook里的指示进行操作。

In [4]:
%%js
var url = window.location.hostname + '/nsight/';
element.append(url)

<IPython.core.display.Javascript object>

单击 _Connect_ 按钮后，远程桌面将要求您输入密码，即`nvidia`。

### 打开远程桌面终端应用程序

接下来，单击位于远程桌面屏幕底部的命令行终端应用程序图标：

![terminal](images/terminal.png)

### 打开Nsight Systems

要打开Nsight Systems，请从现在打开的命令行终端中输入并运行`nsight-sys`（注意没有空格）命令：

![open nsight](images/open-nsight.png)

### 启用使用情况报告

出现提示时，单击“Yes”以启用使用情况报告：

![enable usage](images/enable_usage.png)

### 打开报告文件

通过从Nsight Systems菜单访问 _File_-> _Open_ 来打开此报告文件，然后转到路径 `/root/Desktop/reports`并选择`vector-add-no-prefetch-report.qdrep`。 您在本实验中生成的所有报告都将位于此`root/Desktop/reports`目录中：

![open-report](images/open-report.png)

### 忽略警告/错误信息

您可以关闭并忽略看到的所有警告或错误，这仅仅是我们特定的远程桌面环境产生的结果：

![ignore errors](images/ignore-error.png)

### 为活动时间表腾出更多的显示空间

为了使您的体验更好，可全屏显示性能分析结果，即关闭 _Project Explorer_ 并隐藏 _Events View_ ：

![make nice](images/make-nice.png)

屏幕现在应如下所示：

![now nice](images/now-nice.png)

### 扩展CUDA统一内存的活动时间表

接下来，展开 _CUDA_-> _Unified memory_ 和 _Context_ 时间表，并关闭 _OS运行时库_ 的时间表：

![open memory](images/open-memory.png)

### 观察到有很多次的内存传输操作

略看一下即知，您的应用程序运行大约需要1秒钟，并且在运行`addVectorsInto`核函数期间，还有很多的UM内存活动：

![memory and kernel](images/memory-and-kernel.png)

放大内存的时间轴，可以更清楚地看到由按需内存页面错误引起的所有小内存传输。 几个提示：

1. 您可以在滚动鼠标/触控板的同时按住`Ctrl`来放大和缩小时间轴的任何一点
2. 您可以通过以下方式放大任意部分：单击+拖动它周围的矩形，然后选择 _Zoom in_

这是放大查看许多小的内存传输的示例：

![many transfers](images/many-transfers.png)

---
## 用Nsight Systems重复比较不同的代码重构后的性能

现在，您已经启动并运行了Nsight Systems，并且可以轻松地在活动时间表里移动位置。接下来，您将对您使用已经熟悉的技术进行了迭代改进的程序进行性能分析。 每做一次性能分析，活动时间表都会对下一步如何修改代码提供有用的信息。 这样做还将使您更深地理解各种CUDA编程技术如何影响应用程序的性能。

### 练习：比较预取与不预取的活动时间表

[`01-vector-add-prefetch-solution.cu`](../edit/01-vector-add/solutions/01-vector-add-prefetch-solution.cu)对上面的向量加法程序进行重构，以便它的`addVectorsInto`核函数所需的3个向量在核函数被调用之前就被异步预取到活动的GPU设备上（使用[`cudaMemPrefetchAsync`](http://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1ge8dc9199943d421bc8bc7f473df12e42)）。 请打开源代码，并确定这些更改在应用程序中的何处进行的。

在检查了更改之后，使用下方的代码执行单元来编译并运行重构的应用程序。 您应该看到打印出来的成功消息。

In [5]:
!nvcc -o vector-add-prefetch 01-vector-add/solutions/01-vector-add-prefetch-solution.cu -run

Success! All values calculated correctly.


现在为该版本的应用程序创建一个报告文件：

In [6]:
!nsys profile --stats=true -o vector-add-prefetch-report ./vector-add-prefetch


**** collection configuration ****
	output_filename = /dli/task/vector-add-prefetch-report
	force-overwrite = false
	stop-on-exit = true
	export_sqlite = true
	stats = true
	capture-range = none
	stop-on-range-end = false
	Beta: ftrace events:
	ftrace-keep-user-config = false
	trace-GPU-context-switch = false
	delay = 0 seconds
	duration = 0 seconds
	kill = signal number 15
	inherit-environment = true
	show-output = true
	trace-fork-before-exec = false
	sample_cpu = true
	backtrace_method = LBR
	wait = all
	trace_cublas = false
	trace_cuda = true
	trace_cudnn = false
	trace_nvtx = true
	trace_mpi = false
	trace_openacc = false
	trace_vulkan = false
	trace_opengl = true
	trace_osrt = true
	osrt-threshold = 0 nanoseconds
	cudabacktrace = false
	cudabacktrace-threshold = 0 nanoseconds
	profile_processes = tree
	application command = ./vector-add-prefetch
	application arguments = 
	application working directory = /dli/task
	NVTX profiler range trigger = 
	NVTX profiler domain trigger = 
	

在Nsight Systems中打开报告，让先前的报告保持打开的状态以进行比较。

- 在添加异步预取之前，执行时间与`addVectorsInto`核函数的执行时间相比如何？
- 在活动时间表的 *CUDA API* 部分中找到`cudaMemPrefetchAsync`。
- 内存传输有什么变化？

### 练习：使用核函数进行向量初始化并分析其性能

在向量加法程序的先前迭代中，向量数据是在CPU上初始化的，因此需要在addVectorsInto核函数对其进行操作之前将其迁移到GPU上。

该应用程序的下一个迭代[01-init-kernel-solution.cu](../edit/02-init-kernel/solutions/01-init-kernel-solution.cu)已重构为在GPU上并行处理初始化数据。

由于初始化现是在GPU上进行的，因此预取要在初始化之前完成的，而不是在向量加法之前完成的。 查看源代码以识别在何处进行了这些更改。

在检查了更改之后，使用下方的代码执行单元来编译并运行重构的应用程序。 您应该看到打印成功消息。

In [7]:
!nvcc -o init-kernel 02-init-kernel/solutions/01-init-kernel-solution.cu -run

Success! All values calculated correctly.


现在为该版本的应用程序创建一个报告文件：

In [8]:
!nsys profile --stats=true -o init-kernel-report ./init-kernel


**** collection configuration ****
	output_filename = /dli/task/init-kernel-report
	force-overwrite = false
	stop-on-exit = true
	export_sqlite = true
	stats = true
	capture-range = none
	stop-on-range-end = false
	Beta: ftrace events:
	ftrace-keep-user-config = false
	trace-GPU-context-switch = false
	delay = 0 seconds
	duration = 0 seconds
	kill = signal number 15
	inherit-environment = true
	show-output = true
	trace-fork-before-exec = false
	sample_cpu = true
	backtrace_method = LBR
	wait = all
	trace_cublas = false
	trace_cuda = true
	trace_cudnn = false
	trace_nvtx = true
	trace_mpi = false
	trace_openacc = false
	trace_vulkan = false
	trace_opengl = true
	trace_osrt = true
	osrt-threshold = 0 nanoseconds
	cudabacktrace = false
	cudabacktrace-threshold = 0 nanoseconds
	profile_processes = tree
	application command = ./init-kernel
	application arguments = 
	application working directory = /dli/task
	NVTX profiler range trigger = 
	NVTX profiler domain trigger = 
	environment vari


Generating NVTX Push-Pop Range Statistics...
NVTX Push-Pop Range Statistics (nanoseconds)






在Nsight Systems中打开新的报告文件，然后执行以下操作：

- 将应用程序和`addVectorsInto`运行时间与应用程序的先前版本进行比较，有什么变化？
- 查看时间表中的 *Kernels* 部分。 这两个核函数（`addVectorsInto`和初始化核函数）中的哪一个占用了GPU的大部分时间？
- 您的应用程序包含以下哪项？
   - 数据迁移（HtoD）
   - 数据迁移（DtoH）

### 练习：使用异步预将数据取回主机，并分析其性能

目前，向量加法应用程序是在主机上验证向量加法核函数的结果，所以在该应用程序的下一个重构[01-prefetch-check-solution.cu](../edit/04-prefetch-check/solutions/01-prefetch-check-solution.cu)中，我们用异步预取将数据取回主机后再进行验证。

在检查了更改之后，使用下方的代码执行单元来编译并运行重构的应用程序。 您应该看到打印出来的成功消息。

In [9]:
!nvcc -o prefetch-to-host 04-prefetch-check/solutions/01-prefetch-check-solution.cu -run

Success! All values calculated correctly.


现在为该版本的应用程序创建一个报告文件：

In [10]:
!nsys profile --stats=true -o prefetch-to-host-report ./prefetch-to-host


**** collection configuration ****
	output_filename = /dli/task/prefetch-to-host-report
	force-overwrite = false
	stop-on-exit = true
	export_sqlite = true
	stats = true
	capture-range = none
	stop-on-range-end = false
	Beta: ftrace events:
	ftrace-keep-user-config = false
	trace-GPU-context-switch = false
	delay = 0 seconds
	duration = 0 seconds
	kill = signal number 15
	inherit-environment = true
	show-output = true
	trace-fork-before-exec = false
	sample_cpu = true
	backtrace_method = LBR
	wait = all
	trace_cublas = false
	trace_cuda = true
	trace_cudnn = false
	trace_nvtx = true
	trace_mpi = false
	trace_openacc = false
	trace_vulkan = false
	trace_opengl = true
	trace_osrt = true
	osrt-threshold = 0 nanoseconds
	cudabacktrace = false
	cudabacktrace-threshold = 0 nanoseconds
	profile_processes = tree
	application command = ./prefetch-to-host
	application arguments = 
	application working directory = /dli/task
	NVTX profiler range trigger = 
	NVTX profiler domain trigger = 
	enviro

在Nsight Systems中打开此报告文件，然后执行以下操作：

- 使用时间表里的 *Unified Memory*部分，对比向CPU添加预取之前和之后的*Data Migration (DtoH)*事件。

---
## 并发CUDA流


您现在将学习一个新概念 **CUDA Streams**。 在对它们进行介绍之后，您将返回使用Nsight Systems来更好地评估它们对应用程序性能的影响。

以下幻灯片以可视化的方式概要地介绍了即将学习的内容。请在进入以下更详细的内容之前浏览每一页幻灯片。

In [ ]:
%%HTML

<div align="center"><iframe src="https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-01-V1/AC_STREAMS_NVVP-zh/NVVP-Streams-1-zh.pptx" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe></div>

在 CUDA 编程中，**流**是由按顺序执行的一系列命令构成。在 CUDA 应用程序中，核函数的执行以及一些内存传输均在 CUDA 流中进行。不过直至此时，您仍未直接与 CUDA 流打交道；但实际上您的 CUDA 代码已在名为*默认流*的流中执行了其核函数。

除默认流以外，CUDA 程序员还可创建并使用非默认 CUDA 流，此举可支持执行多个操作，例如在不同的流中并发执行多个核函数。多流的使用可以为您的加速应用程序带来另外一个层次的并行，并能提供更多应用程序的优化机会。

### 控制CUDA流行为的规则

为有效利用 CUDA 流，您应了解有关 CUDA 流行为的以下几项规则：

- 给定流中的所有操作会按序执行。
- 就不同非默认流中的操作而言，无法保证其会按彼此之间的任何特定顺序执行。
- 默认流具有阻断能力，即，它会等待其它已在运行的所有流完成当前操作之后才运行，但在其自身运行完毕之前亦会阻碍其它流的运行。

### 创建，使用和销毁非默认CUDA流

以下代码段演示了如何创建，利用和销毁非默认CUDA流。您会注意到，要在非默认CUDA流中启动CUDA核函数，必须将流作为执行配置的第4个可选参数传递给该核函数。到目前为止，您仅利用了执行配置的前两个参数：

``` C++
cudaStream_t stream;   // CUDA流的类型为 `cudaStream_t`
cudaStreamCreate(&stream); // 注意，必须将一个指针传递给 `cudaCreateStream`

someKernel<<<number_of_blocks, threads_per_block, 0, stream>>>();   // `stream` 作为第4个EC参数传递

cudaStreamDestroy(stream); // 注意，将值（而不是指针）传递给 `cudaDestroyStream`
```

但值得一提的是，执行配置的第3个可选参数超出了本实验的范围。此参数允许程序员提供**共享内存**（当前将不涉及的高级主题）中为每个内核启动动态分配的字节数。每个块分配给共享内存的默认字节数为“0”，在本练习的其余部分中，您将传递“ 0”作为该值，以便展示我们感兴趣的第4个参数。

### 练习：预测默认流行为

[01-print-numbers](../edit/05-stream-intro/01-print-numbers.cu) 应用程序带有一个非常简单的 `printNumber` 核函数，可用于接受及打印整数。仅在单个线程块内使用单线程执行该核函数，但使用“for 循环”可执行 5 次，并且每次启动时都会传递“for 循环”的迭代次数。

使用下方的代码执行线程块编译和运行 [01-print-numbers](../edit/05-stream-intro/01-print-numbers.cu) 应用程序。您应能看到打印的数字 `0` 至 `4`。

In [11]:
!nvcc -o print-numbers 05-stream-intro/01-print-numbers.cu -run

0
1
2
3
4


既已了解核函数在默认情况下会在默认流中执行，那么据您预计，`print-numbers` 程序的 5 次启动将会顺次执行还是会并行执行？您应能提及默认流的两个功能来支持您的回答。在下面的单元格中创建一个报告文件，然后在Nsight Systems中将其打开以确认您的答案。

In [12]:
!nsys profile --stats=true -o print-numbers-report ./print-numbers


**** collection configuration ****
	output_filename = /dli/task/print-numbers-report
	force-overwrite = false
	stop-on-exit = true
	export_sqlite = true
	stats = true
	capture-range = none
	stop-on-range-end = false
	Beta: ftrace events:
	ftrace-keep-user-config = false
	trace-GPU-context-switch = false
	delay = 0 seconds
	duration = 0 seconds
	kill = signal number 15
	inherit-environment = true
	show-output = true
	trace-fork-before-exec = false
	sample_cpu = true
	backtrace_method = LBR
	wait = all
	trace_cublas = false
	trace_cuda = true
	trace_cudnn = false
	trace_nvtx = true
	trace_mpi = false
	trace_openacc = false
	trace_vulkan = false
	trace_opengl = true
	trace_osrt = true
	osrt-threshold = 0 nanoseconds
	cudabacktrace = false
	cudabacktrace-threshold = 0 nanoseconds
	profile_processes = tree
	application command = ./print-numbers
	application arguments = 
	application working directory = /dli/task
	NVTX profiler range trigger = 
	NVTX profiler domain trigger = 
	environment 

### 练习：实现并发CUDA流

由于核函数的所有 5 次启动均在同一个流中发生，因此看到 5 个核函数顺次执行也就不足为奇。此外，也可以这么说，由于默认流具有阻断作用，所以核函数都会在完成本次启动之后才启动下一次，而事实也是如此。

请重构 [01-print-numbers](../edit/05-stream-intro/01-print-numbers.cu) 应用程序，以便核函数的每次启动都在自身非默认流中进行。若已不再需要所创建的流，请务必予以销毁。请使用下方的代码执行单元编译和运行经重构的代码。您应仍能看到打印的数字 `0` 至 `4`，不过这些数字不一定会按升序排列。如您遇到问题，请参阅 [解决方案](../edit/05-stream-intro/solutions/01-print-numbers-solution.cu)。

In [13]:
!nvcc -o print-numbers-in-streams 05-stream-intro/01-print-numbers.cu -run

0
1
2
3
4


您既已为核函数的 5 次启动使用 5 个不同的非默认流，您预计它们会顺次执行还是会并行执行？除目前对流的了解之外，您还需考虑 `printNumber` 核函数的简单程度，也就是说，即使您预测并行运行，核函数的完成速度是否仍会允许完全重叠？

进行假设后，在Nsight Systems中打开一个新的报告文件以查看其实际行为。 您应该注意到，现在，_CUDA_ 部分中为您创建的每个非默认流提供了其他行：

In [14]:
!nsys profile --stats=true -o print-numbers-in-streams-report print-numbers-in-streams


**** collection configuration ****
	output_filename = /dli/task/print-numbers-in-streams-report
	force-overwrite = false
	stop-on-exit = true
	export_sqlite = true
	stats = true
	capture-range = none
	stop-on-range-end = false
	Beta: ftrace events:
	ftrace-keep-user-config = false
	trace-GPU-context-switch = false
	delay = 0 seconds
	duration = 0 seconds
	kill = signal number 15
	inherit-environment = true
	show-output = true
	trace-fork-before-exec = false
	sample_cpu = true
	backtrace_method = LBR
	wait = all
	trace_cublas = false
	trace_cuda = true
	trace_cudnn = false
	trace_nvtx = true
	trace_mpi = false
	trace_openacc = false
	trace_vulkan = false
	trace_opengl = true
	trace_osrt = true
	osrt-threshold = 0 nanoseconds
	cudabacktrace = false
	cudabacktrace-threshold = 0 nanoseconds
	profile_processes = tree
	application command = print-numbers-in-streams
	application arguments = 
	application working directory = /dli/task
	NVTX profiler range trigger = 
	NVTX profiler domain trig

![stream print](images/streams-print.png)

### 练习：将流用于并行进行数据初始化的核函数

您一直使用的向量加法应用程序 [01-prefetch-check-solution.cu](../edit/04-prefetch-check/solutions/01-prefetch-check-solution.cu) 目前启动 3 次初始化核函数，即：为 `vectorAdd` 核函数需要初始化的 3 个向量分别启动一次。重构该应用程序，以便在其各自的非默认流中启动全部 3 个初始化核函数。在使用下方的代码执行单元编译及运行时，您应仍能看到打印的成功消息。如您遇到问题，请参阅 [解决方案](../edit/06-stream-init/solutions/01-stream-init-solution.cu)。

In [15]:
!nvcc -o init-in-streams 04-prefetch-check/solutions/01-prefetch-check-solution.cu -run

Success! All values calculated correctly.


在Nsight Systems中打开一个报告，以确认您的3个初始化内核启动正在它们自己的非默认流中运行，并且存在一定程度的并发重叠。

In [16]:
!nsys profile --stats=true -o init-in-streams-report ./init-in-streams


**** collection configuration ****
	output_filename = /dli/task/init-in-streams-report
	force-overwrite = false
	stop-on-exit = true
	export_sqlite = true
	stats = true
	capture-range = none
	stop-on-range-end = false
	Beta: ftrace events:
	ftrace-keep-user-config = false
	trace-GPU-context-switch = false
	delay = 0 seconds
	duration = 0 seconds
	kill = signal number 15
	inherit-environment = true
	show-output = true
	trace-fork-before-exec = false
	sample_cpu = true
	backtrace_method = LBR
	wait = all
	trace_cublas = false
	trace_cuda = true
	trace_cudnn = false
	trace_nvtx = true
	trace_mpi = false
	trace_openacc = false
	trace_vulkan = false
	trace_opengl = true
	trace_osrt = true
	osrt-threshold = 0 nanoseconds
	cudabacktrace = false
	cudabacktrace-threshold = 0 nanoseconds
	profile_processes = tree
	application command = ./init-in-streams
	application arguments = 
	application working directory = /dli/task
	NVTX profiler range trigger = 
	NVTX profiler domain trigger = 
	environm

---
## 总结

在本实验中，您学习了：

- 使用 **Nsight Systems** 直观地描述GPU加速的CUDA应用程序的活动时间表。
- 使用Nsight Systems识别和利用GPU加速的CUDA应用程序中的优化机会。
- 利用CUDA流在被加速的应用程序中并发执行核函数。

此时，您拥有大量的基本工具和技术，可用于加速原本只能在CPU上的应用程序，然后不断优化那些被加速的应用程序。 在最后的练习中，您将有机会应用学到的所有知识来加速[n-body](https://en.wikipedia.org/wiki/N-body_problem)模拟器，以预测受引力互相影响的一组物体的集合。

---
## 最后的练习：加速和优化N体模拟器

[n-body](https://en.wikipedia.org/wiki/N-body_problem) 模拟器可以预测通过引力相互作用的一组物体的个体运动。[01-nbody.cu](../edit/09-nbody/01-nbody.cu) 包含一个简单而有效的 n-body 模拟器，适合用于在三维空间移动的物体。我们可通过向该应用程序传递一个命令行参数以影响系统中的物体数量。

以目前的仅用CPU的情况下，此应用程序大约需要5秒钟才能运行4096个物体，需要**20分钟**才能运行65536个物体。您的任务是用GPU加速程序，同时保持仿真的正确性。

### 做此练习时可考虑的注意事项

在开始任务之前，请注意以下事项：

- 在第一次重构中，请格外注意应用程序的逻辑部分（尤其是 `bodyForce` 函数）能够并且应该基本保持不变：要侧重于尽可能轻松地加速应用程序。
- 代码库包含 `main` 函数内的“for 循环”，用于将 `bodyForce` 函数计算的物体间的引力集成到系统中每个物体的所在位置。该集成不仅需在 `bodyForce` 函数运行后进行，并且需在下一次调用 `bodyForce` 函数之前完成。在选择并行化的处理方式和程序位置时，请牢记这一点。
- 使用基于性能分析驱动和迭代的方法。
- 无需为代码添加错误处理，但其可能有助您确保代码的顺利运行。

快去开心地执行任务吧！

使用此单元格来编译nbody模拟器。尽管它最初只是一个仅用于CPU的应用程序，但却可以准确地模拟物体的位置。

In [17]:
!nvcc -std=c++11 -o nbody 09-nbody/01-nbody.cu

强烈建议您使用性能分析器来协助您的工作。执行以下单元格以生成性能分析报告文件：

In [18]:
!nsys profile --stats=true --force-overwrite=true -o nbody-report ./nbody


**** collection configuration ****
	output_filename = /dli/task/nbody-report
	force-overwrite = true
	stop-on-exit = true
	export_sqlite = true
	stats = true
	capture-range = none
	stop-on-range-end = false
	Beta: ftrace events:
	ftrace-keep-user-config = false
	trace-GPU-context-switch = false
	delay = 0 seconds
	duration = 0 seconds
	kill = signal number 15
	inherit-environment = true
	show-output = true
	trace-fork-before-exec = false
	sample_cpu = true
	backtrace_method = LBR
	wait = all
	trace_cublas = false
	trace_cuda = true
	trace_cudnn = false
	trace_nvtx = true
	trace_mpi = false
	trace_openacc = false
	trace_vulkan = false
	trace_opengl = true
	trace_osrt = true
	osrt-threshold = 0 nanoseconds
	cudabacktrace = false
	cudabacktrace-threshold = 0 nanoseconds
	profile_processes = tree
	application command = ./nbody
	application arguments = 
	application working directory = /dli/task
	NVTX profiler range trigger = 
	NVTX profiler domain trigger = 
	environment variables:

The t

在这里，我们导入一个函数，该函数将针对各种数量的物体运行您的`nbody`模拟器，以检查其性能和准确性。

In [19]:
from assessment import run_assessment

执行以下单元格以运行并评估`nbody`：

In [20]:
run_assessment()

使用4096个物体运行n-体模拟器
----------------------------------------

此应用程序应该运行得快于0.9秒。
您的应用程序运行了: 0.2686秒
您的应用程序运行速度是  
您的结果是正确的。

使用65536个物体运行n-体模拟器
----------------------------------------

此应用程序应该运行得快于1.3秒。
您的应用程序运行了: 0.6128秒
您的应用程序运行速度是  
您的结果是正确的。

祝贺您！您通过了评估！
请参阅下面的说明来生成证书，并查看您是否可以进一步加速模拟器！


## 生成证书

如果您通过了评估，请返回课程页面（如下所示）并单击"ASSESS TASK"（评估任务）按钮，这将为您生成该课程的证书。

![run_assessment](./images/run_assessment.png)

## 进阶内容

以下章节专为时间富余和有意深究的学习者而设，其中将介绍更多中级技术，其中会涉及部分手动内存管理，以及使用非默认流重叠执行核函数和内存拷贝。

在了解以下所列的各项技术后，您可尝试运用这些技术进一步优化 n-body 模拟器。

---
## 手动内存分配和复制

尽管 `cudaMallocManaged` 和 `cudaMemPrefetchAsync` 函数性能出众并能大幅简化内存迁移，但有时也有必要使用更多手动内存分配方法。这在已知只需在设备或主机上访问数据时尤其如此，并且因免于进行自动按需迁移而能够收回数据迁移成本。

此外，通过手动内存管理，您可以使用非默认流同时开展数据传输与计算工作。在本节中，您将学习一些基本的手动内存分配和拷贝技术，之后会延伸应用这些技术以同时开展数据拷贝与计算工作。

以下是一些用于手动内存管理的 CUDA 命令：

- `cudaMalloc` 命令将直接为处于活动状态的 GPU 分配内存。这可防止出现所有 GPU 分页错误，而代价是主机代码将无法访问该命令返回的指针。
- `cudaMallocHost` 命令将直接为 CPU 分配内存。该命令可“固定”内存(pinned memory)或“页锁定”内存(page-locked memory)，此举允许将内存异步拷贝至 GPU 或从 GPU 异步拷贝至内存。固定内存过多则会干扰 CPU 性能，因此请勿无端使用该命令。释放固定内存时应使用 `cudaFreeHost` 命令。
- 无论是从主机到设备还是从设备到主机，`cudaMemcpy` 命令均可拷贝（而非传输）内存。

### 手动内存管理示例

以下是一段演示使用上述 CUDA API 调用的代码。

```cpp
int *host_a, *device_a;        // Define host-specific and device-specific arrays.
cudaMalloc(&device_a, size);   // `device_a` is immediately available on the GPU.
cudaMallocHost(&host_a, size); // `host_a` is immediately available on CPU, and is page-locked, or pinned.

initializeOnHost(host_a, N);   // No CPU page faulting since memory is already allocated on the host.

// `cudaMemcpy` takes the destination, source, size, and a CUDA-provided variable for the direction of the copy.
cudaMemcpy(device_a, host_a, size, cudaMemcpyHostToDevice);

kernel<<<blocks, threads, 0, someStream>>>(device_a, N);

// `cudaMemcpy` can also copy data from device to host.
cudaMemcpy(host_a, device_a, size, cudaMemcpyDeviceToHost);

verifyOnHost(host_a, N);

cudaFree(device_a);
cudaFreeHost(host_a);          // Free pinned memory like this.
```

### 练习：手动分配主机和设备内存

向量加法应用程序[01-stream-init-solution](../edit/06-stream-init/solutions/01-stream-init-solution.cu)的最新迭代使用 `cudaMallocManaged` 命令分配受管理的内存，该内存首先由初始化核函数在设备上使用，然后由向量加法核函数在设备上使用，然后再由主机所使用以对加法结果做验证。此时内存数据的传输是自动进行的。这是种很明智的方法，但我们也值得尝试一些手动内存分配和拷贝方法，以观察其对应用程序性能的影响。

将 [01-stream-init-solution](../edit/06-stream-init/solutions/01-stream-init-solution.cu) 应用程序重构为**不**使用 `cudaMallocManaged` 命令。为此，您需要执行以下操作：

- 将调用 `cudaMallocManaged` 命令替换为调用 `cudaMalloc` 命令。
- 创建将用于在主机上验证的额外向量。这是由于使用 `cudaMalloc` 命令分配的内存在主机上不可用，因此您必须执行此操作, 使用 `cudaMallocHost` 命令分配此主机向量。
- 在 `addVectorsInto` 核函数运行完毕后，使用 `cudaMemcpy` 命令将包含相加结果的向量复制到使用 `cudaMallocHost` 命令创建的主机向量中。
- 使用 `cudaFreeHost` 命令释放经由 `cudaMallocHost` 命令分配的内存。

如您遇到问题，请参阅 [解决方案](../edit/07-manual-malloc/solutions/01-manual-malloc-solution.cu) 。

In [21]:
!nvcc -o vector-add-manual-alloc 06-stream-init/solutions/01-stream-init-solution.cu -run

Success! All values calculated correctly.


完成重构后，在Nsight Systems中打开一个报告，并使用活动时间表执行以下操作：

- 注意，时间表的*统一内存*部分将不复存在。
- 比较此时间表与之前重构的时间表，并使用时间轴标尺比较当前应用程序中 `cudaMalloc` 的运行时间与先前应用程序中 `cudaMallocManaged` 的运行时间。
- 查看当前应用程序中初始化核函数的运行开始时间如何会晚于其在上次迭代中的运行时间。检查过时间轴后，您将发现时间差在于 `cudaMallocHost` 命令所用的时间。这很清楚地表明内存传输与内存拷贝的区别。正如您当前的操作，拷贝内存时，数据将存在于系统中的 2 个不同位置。与在上次迭代中仅分配 3 个向量相比，当前分配第 4 个主机向量会产生较小的性能成本。

---
## 使用流实现数据传输和代码的重叠执行

以下幻灯片概要地讲解了后面将涉及的内容。请点击浏览一遍这些幻灯片，然后再继续深入了解以下章节中的主题。

In [ ]:
%%HTML

<div align="center"><iframe src="https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-01-V1/AC_STREAMS_NVVP-zh/NVVP-Streams-3-zh.pptx" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe></div>

除了`cudaMemcpy`之外，还有`cudaMemcpyAsync`，只要固定了主机内存，它就可以从主机到设备或从设备到主机异步复制内存，这可以通过使用`cudaMallocHost`分配它来完成。

与核函数的执行类似，`cudaMemcpyAsync`在默认情况下仅相对于主机是异步的。默认情况下，它在默认流中执行，因此对于GPU上发生的其他CUDA操作而言，它是阻塞操作。但是，`cudaMemcpyAsync`函数将非默认流作为可选的第5个参数。通过向其传递非默认流，可以将内存传输与其他非默认流中发生的其他CUDA操作并发。

一种常见且有用的模式是结合使用固定主机内存，非默认流中的异步内存副本和非默认流中的核函数执行，以使内存传输与核函数的执行重叠。

在以下示例中，我们并非在等待整个内存拷贝完成之后再开始运行核函数，而是拷贝并处理所需的数据段，并让每个拷贝/处理中的数据段均在各自的非默认流中运行。通过使用此技术，您可以开始处理部分数据，同时为后续段并发执行内存传输。使用此技术计算操作次数的数据段特定值和数组内的偏移位置时必须格外小心，如下所示：

```cpp
int N = 2<<24;
int size = N * sizeof(int);

int *host_array;
int *device_array;

cudaMallocHost(&host_array, size);               // Pinned host memory allocation.
cudaMalloc(&device_array, size);                 // Allocation directly on the active GPU device.

initializeData(host_array, N);                   // Assume this application needs to initialize on the host.

const int numberOfSegments = 4;                  // This example demonstrates slicing the work into 4 segments.
int segmentN = N / numberOfSegments;             // A value for a segment's worth of `N` is needed.
size_t segmentSize = size / numberOfSegments;    // A value for a segment's worth of `size` is needed.

// For each of the 4 segments...
for (int i = 0; i < numberOfSegments; ++i)
{
  // Calculate the index where this particular segment should operate within the larger arrays.
  segmentOffset = i * segmentN;

  // Create a stream for this segment's worth of copy and work.
  cudaStream_t stream;
  cudaStreamCreate(&stream);
  
  // Asynchronously copy segment's worth of pinned host memory to device over non-default stream.
  cudaMemcpyAsync(&device_array[segmentOffset],  // Take care to access correct location in array.
                  &host_array[segmentOffset],    // Take care to access correct location in array.
                  segmentSize,                   // Only copy a segment's worth of memory.
                  cudaMemcpyHostToDevice,
                  stream);                       // Provide optional argument for non-default stream.
                  
  // Execute segment's worth of work over same non-default stream as memory copy.
  kernel<<<number_of_blocks, threads_per_block, 0, stream>>>(&device_array[segmentOffset], segmentN);
  
  // `cudaStreamDestroy` will return immediately (is non-blocking), but will not actually destroy stream until
  // all stream operations are complete.
  cudaStreamDestroy(stream);
}
```

### 练习：核函数和内存复制回主机重叠执行

向量加法应用程序 [01-manual-malloc-solution.cu](../edit/07-manual-malloc/solutions/01-manual-malloc-solution.cu) 的最新迭代目前正在 GPU 上执行所有向量加法操作，完成后其会将内存拷贝回主机以进行验证。

重构 [01-manual-malloc-solution.cu](../edit/07-manual-malloc/solutions/01-manual-malloc-solution.cu) 应用程序，在非默认流中分4段执行矢量加法，以便可以在等待所有向量加法工作完成之前开始异步内存复制。如您遇到问题，请参阅 [解决方案](../edit/08-overlap-xfer/solutions/01-overlap-xfer-solution.cu)。

In [25]:
!nvcc -o vector-add-manual-alloc 07-manual-malloc/solutions/01-manual-malloc-solution.cu -run
!tar zcvf task.gz ../task

Success! All values calculated correctly.
assessment  assessment_results	task


完成重构后，在Nsight Systems中打开一个报告，并使用活动时间表执行以下操作：

- 记录设备到主机的内存传输开始时间是在所有核函数工作完成之前还是之后？
- 需注意 4 个内存拷贝段本身并不重叠。即使是在单独的非默认流中，在给定方向（此处为 DtoH）上，每次也只能同时进行一个内存传输。此处获得性能提升的原因在于其能先于其他方式开始内存传输，并且不难想象：若在某个应用程序中所完成的工作量与简单的加法运算相比，并非可以几乎忽略不计，那么内存拷贝不仅开始得更早，而且还会与核函数的执行相重叠。